In [1]:
from qibo import set_backend, get_backend
from qibo.backends import Modality
from qibo.gates import PS, BS, H
from qibo.models import Circuit
import numpy as np

In [2]:
set_backend("loqc")

[Qibo 0.2.19|INFO|2025-06-30 16:16:43]: Using LOQC strong simulation backend on /CPU:0


In [3]:
backend = get_backend()
print(backend.versions, backend.modality)

{'qibo': '0.2.19', 'perceval': '0.13.0'} Modality.PHOTONIC_CV


In [4]:
# Retrieve matrix for parametrized gates
bs1 = BS(0, 1, np.pi/2)
print("theta=pi/2", bs1.matrix())
print(bs1.photonic_component.compute_unitary())

bs1.parameters = [0.4]
print("theta=0.4", bs1.matrix())
print(bs1.photonic_component.compute_unitary())

# Through the backend too
print("Phase shifter, phi=0.3", backend.matrix(PS(0, 0.3)))

theta=pi/2 [[0.70710678+0.j         0.        -0.70710678j]
 [0.        -0.70710678j 0.70710678+0.j        ]]
[[0.70710678+0.j         0.        +0.70710678j]
 [0.        +0.70710678j 0.70710678+0.j        ]]
theta=0.4 [[0.98006658+0.j         0.        -0.19866933j]
 [0.        -0.19866933j 0.98006658+0.j        ]]
[[0.98006658+0.j         0.        +0.19866933j]
 [0.        +0.19866933j 0.98006658+0.j        ]]
Phase shifter, phi=0.3 [-0.41614684+0.90929743j]


In [5]:
# Create a photonic circuit

pc = Circuit(3, modality=Modality.PHOTONIC)
pc.add(H(0))
pc.add(PS(0, 0.3, trainable=True))
pc.add(BS(0, 2))
pc.add(BS(1, 2))
pc.add(PS(2, 0.8, trainable=False))
pc.add(BS(1, 2))


In [6]:
print(pc.summary())
print()
print(pc.diagram())

Circuit depth = 6
Total number of gates = 6
Number of modes = 3
Most common gates:
Beam splitter: 3
Phase shifter: 2
h: 1

0: ─H─PS─X────────
1: ──────|─X────X─
2: ──────X─X─PS─X─


In [7]:
print(pc.to_pcvl())

pcvl.Experiment(3)\
    .add((0, 1), BS.H())\
    .add((0,), PS(0.3))\
    .add((0, 2), BS(1.5707963267948966))\
    .add((1, 2), BS(1.5707963267948966))\
    .add((2,), PS(0.8))\
    .add((1, 2), BS(1.5707963267948966))


In [8]:
# circuit simulation
params = [0.]
res = pc(initial_state=(1, 0, 1), nshots=100)
print(res)

MeasurementResult(qubits=3, nshots=100)


In [9]:
res.samples()

[[0, 1, 1],
 [2, 0, 0],
 [1, 0, 1],
 [0, 0, 2],
 [1, 1, 0],
 [1, 1, 0],
 [2, 0, 0],
 [2, 0, 0],
 [0, 1, 1],
 [2, 0, 0],
 [0, 2, 0],
 [0, 0, 2],
 [0, 1, 1],
 [2, 0, 0],
 [1, 1, 0],
 [0, 2, 0],
 [2, 0, 0],
 [1, 1, 0],
 [0, 2, 0],
 [0, 0, 2],
 [0, 0, 2],
 [2, 0, 0],
 [1, 1, 0],
 [0, 1, 1],
 [0, 1, 1],
 [0, 1, 1],
 [0, 0, 2],
 [1, 1, 0],
 [1, 1, 0],
 [1, 1, 0],
 [2, 0, 0],
 [1, 1, 0],
 [0, 0, 2],
 [2, 0, 0],
 [1, 1, 0],
 [2, 0, 0],
 [2, 0, 0],
 [2, 0, 0],
 [1, 0, 1],
 [1, 1, 0],
 [0, 1, 1],
 [1, 1, 0],
 [0, 0, 2],
 [0, 1, 1],
 [0, 0, 2],
 [1, 0, 1],
 [0, 0, 2],
 [0, 2, 0],
 [0, 1, 1],
 [0, 1, 1],
 [1, 1, 0],
 [0, 0, 2],
 [0, 2, 0],
 [2, 0, 0],
 [0, 1, 1],
 [0, 1, 1],
 [0, 0, 2],
 [0, 0, 2],
 [1, 1, 0],
 [1, 1, 0],
 [0, 0, 2],
 [1, 1, 0],
 [1, 1, 0],
 [1, 1, 0],
 [0, 0, 2],
 [2, 0, 0],
 [2, 0, 0],
 [1, 1, 0],
 [0, 1, 1],
 [0, 1, 1],
 [2, 0, 0],
 [1, 1, 0],
 [0, 0, 2],
 [0, 1, 1],
 [0, 1, 1],
 [0, 2, 0],
 [0, 0, 2],
 [0, 0, 2],
 [0, 0, 2],
 [0, 1, 1],
 [0, 1, 1],
 [2, 0, 0],
 [0, 1, 1],
 [0,

In [10]:
# Now with a Cloud back-end
set_backend("quandela", platform="qpu:ascella", token="YOUR TOKEN")  # Requires a valid Quandela Cloud token
res = pc(initial_state=(1, 0, 1), nshots=100)
res.samples()  # qpu:ascella is a noisy platform (it's an actual QPU), hence the photon loss observed in the results (nearly all output states have only 1 detection)

[Qibo 0.2.19|INFO|2025-06-30 16:16:43]: Using Quandela Cloud backend on /CPU:0


[[1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [1, 1, 0],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [1, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 1, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 0],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 1, 0],
 [0, 0, 1],
 [1, 0, 0],
 [0, 0, 1],
 [0, 1, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [1, 0, 0],
 [1, 0, 0],
 [1, 0, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 1, 0],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [0, 0, 1],
 [1,